__IMPORTING NECESSARY LIBARIES__

In [1]:
import folium
import wget
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

#### Task 1: Mark all launch sites on a map


In [2]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [3]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [6]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Lat'], site['Long']], color='#d35400', radius=50, fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.Marker(
    [site['Lat'], site['Long']],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

#### Task 2: Mark the success/failed launches for each site on the map

In [7]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [8]:
marker_cluster = MarkerCluster()

In [9]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [10]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    
    marker = folium.Marker([record['Lat'], record['Long']], 
                  icon=folium.Icon(color='white', icon_color = record['marker_color']))
    
    marker_cluster.add_child(marker)
    
site_map

#### TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a <code>MousePosition</code> on the map to get coordinate for a mouse over a point on the map. 
As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

You can calculate the distance between two points on the map based on their <code>Lat</code> and <code>Long</code> values using the following method:



In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [13]:
#CCAFS-LC40
lat_coastline, long_coastline = 28.56198, -80.56774
lat_launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40']['Lat']
long_launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40']['Long']

distance_coastline = calculate_distance(lat_launch_site, long_launch_site, lat_coastline, long_coastline)
print(distance_coastline, 'KM')

0.9401457138766881 KM


After obtained its coordinate, create a <code>folium.Marker</code> to show the distance

In [14]:
distance_marker = folium.Marker(
    [28.56227, -80.56839],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline)
        )
    )

site_map.add_child(distance_marker)

lines=folium.PolyLine(locations = [[lat_coastline, long_coastline],[lat_launch_site, long_launch_site]], weight=3)
site_map.add_child(lines)

### Now lets see if the launch site is in a close proximity to highways, railway stations or cities.This will provide us information about the place selection of launch sites.

In [15]:
# Closest Highway - Benchline Expressway

distance_highway = calculate_distance(lat_launch_site, long_launch_site, 28.41162, -80.82019)
print(distance_highway, 'KM')

distance_marker_highway = folium.Marker(
    [28.41162, -80.82019],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 400; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway)
        )
    )

site_map.add_child(distance_marker_highway)

lines=folium.PolyLine(locations = [[28.41162, -80.82019],[lat_launch_site, long_launch_site]], weight=3)
site_map.add_child(lines)

# Closest Railway Station - Lynx Station

distance_railway = calculate_distance(lat_launch_site, long_launch_site, 28.54838, -81.3812)
print(distance_railway, 'KM')

distance_marker_railway = folium.Marker(
    [28.54838, -81.3812],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 400; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway)
        )
    )

site_map.add_child(distance_marker_railway)

lines=folium.PolyLine(locations = [[28.54838, -81.3812],[lat_launch_site, long_launch_site]], weight=3)
site_map.add_child(lines)

# Closest City - Melbourne

distance_city = calculate_distance(lat_launch_site, long_launch_site, 28.1075, -80.63561)
print(distance_city, 'KM')

distance_marker_city = folium.Marker(
    [28.1075, -80.63561],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 400; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city)
        )
    )

site_map.add_child(distance_marker_city)

lines=folium.PolyLine(locations = [[28.1075, -80.63561],[lat_launch_site, long_launch_site]], weight=3)
site_map.add_child(lines)

29.060264970789053 KM
78.55012480670125 KM
50.90801689522832 KM
